#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [119]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__) # flask database name shortened, variable used later
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database # sql used to access, manipulate database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy() 


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [120]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class Album(db.Model): # "User" is schema title for data, class is being defined
    __tablename__ = 'albums'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True) # class User takes data from db.Model (inheritance)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _artist = db.Column(db.String(255), unique=False, nullable=False)
    _songs = db.Column(db.String(255), unique=False, nullable=False)
    _streams = db.Column(db.String(255), unique=False, nullable=False)
    _release = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, artist, songs, streams, release=datetime.today()): # initialize data, set variables and self values
        self._name = name    # variables with self prefix become part of the object, 
        self._artist = artist
        self._songs = songs
        self._streams = streams
        if isinstance(release, str):  # not a date type     
            release = date=datetime.today()
        self._release = release

    # a name getter method, extracts name from object
    @property # "get" to get values
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter # "update" used to change values later
    def name(self, name):
        self._name = name

        # a name getter method, extracts name from object
    @property # "get" to get values
    def artist(self):
        return self._artist
    
    # a setter function, allows name to be updated after initial object creation
    @artist.setter # "update" used to change values later
    def artist(self, artist):
        self._artist = artist
    
    # a getter method, extracts uid from object
    @property
    def songs(self):
        return self._songs
    
    # a setter function, allows uid to be updated after initial object creation
    @songs.setter
    def songs(self, songs):
        self._songs = songs

        # a getter method, extracts uid from object
    @property
    def streams(self):
        return self._streams
    
    # a setter function, allows uid to be updated after initial object creation
    @songs.setter
    def streams(self, streams):
        self._streams = streams
        
    # check if uid parameter matches user id in object, return boolean
    def is_name(self, name):
        return self._name == name

    
    # dob property is returned as string, a string represents date outside object
    @property
    def release(self):
        release_string = self._release.strftime('%m-%d-%Y')
        return release_string
    
    # dob setter, verifies date type before it is set or default to today
    @release.setter
    def release(self, release):
        if isinstance(release, str):  # not a date type     
            release = date=datetime.today()
        self._release = release
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._release.year - ((today.month, today.day) < (self._release.month, self._release.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self): #create a user
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self): #read existing data
        return {
            "id": self.id,
            "name": self.name,
            "artist": self.artist,
            "songs": self.songs,
            "streams": self.streams,
            "release": self.release,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", artist="", songs="", streams=""): #update existing data with new data
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(artist) > 0:
            self.artist = artist
        if len(songs) > 0:
            self.set_songs(songs)
        if len(streams) > 0:
            self.set_streams(streams)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self): #delete an existing entry
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [121]:
"""Database Creation and Testing """


# Builds working data for testing
def initAlbums():
    with app.app_context():
        """Create database and tables"""
        db.create_all() # add data to db file
        """Tester data for table"""
        u1 = Album(name='ye', artist='Kanye West', songs='7', streams='100000', release=datetime(2018, 1, 1))
        u2 = Album(name='The Melodic Blue', artist='Baby Keem', songs='200000', streams='0', release=datetime(2021, 8, 30))
        u3 = Album(name='Heaven or Hell', artist='Don Toliver', songs='75000', streams='0', release=datetime(2019, 6, 9))
        u4 = Album(name='Afterlyfe', artist='Yeat', songs='15', streams='50000', release=datetime(2023, 1, 5))


        albums = [u1, u2, u3, u4]

        """Builds sample user/note(s) data"""
        for album in albums:
            try: #try code, test for error. if error, error message, otherwise create user
                '''add user to table'''
                object = album.create()
                print(f"Created new name {object.name}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist name {album.name}, or error.")
                
initAlbums()

Created new name ye
Created new name The Melodic Blue
Created new name Heaven or Hell
Created new name Afterlyfe


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [122]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_name(name):
    with app.app_context():
        album = Album.query.filter_by(_name=name).first() #only search through uids
    return album # returns user object

# Check credentials by finding user and verify password
def check_credentials(name):
    # query email and return user record
    album = find_by_name(name)
    if album == None:
        return False
        
#check_credentials("indi")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [123]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    name = input("(Create)Enter the name:")
    album = find_by_name(name)
    try:
        print("Found\n", album.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    artist = input("Enter the artist:")
    songs = input("Enter the songs:")
    streams = input("Enter the streams:")
    
    # Initialize User object before date
    album = Album(name=name, 
                  artist=artist,
                  songs=songs,
                  streams=streams
                )
    
    # create user.dob, fail with today as dob
    release = input("Enter the release date 'YYYY-MM-DD'")
    try:
        album.releaase = datetime.strptime(release, '%Y-%m-%d').date()
    except ValueError:
        album.release = datetime.today()
        print(f"Invalid date {release} require YYYY-mm-dd, date defaulted to {album.release}")
           
    # write object to database
    with app.app_context():
        try:
            object = album.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error name {name}")
        
create()

Created
 {'id': 5, 'name': 'watch', 'artist': 'uzi', 'songs': '1', 'streams': '1', 'release': '03-16-2023', 'age': 0}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [124]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Album.query.all() # search all init(columns)
    json_ready = [album.read() for album in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'ye',
  'artist': 'Kanye West',
  'songs': '7',
  'streams': '7',
  'release': '01-01-2018',
  'age': 5},
 {'id': 2,
  'name': 'The Melodic Blue',
  'artist': 'Baby Keem',
  'songs': '16',
  'streams': '16',
  'release': '01-01-2021',
  'age': 2},
 {'id': 3,
  'name': 'Heaven or Hell',
  'artist': 'Don Toliver',
  'songs': '0',
  'streams': '0',
  'release': '01-01-2021',
  'age': 2},
 {'id': 4,
  'name': 'Afterlyfe',
  'artist': 'Yeat',
  'songs': '0',
  'streams': '0',
  'release': '01-01-2023',
  'age': 0},
 {'id': 5,
  'name': 'watch',
  'artist': 'uzi',
  'songs': '1',
  'streams': '1',
  'release': '03-16-2023',
  'age': 0}]

In [125]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def update():
    # optimize user time to see if uid exists
    name = input("(update)Enter the name:")
    album = find_by_name(name)
    try:
        print("Found\n", album.read())
        pass
    except:
        print("invalid album")
        return
    
    # request value that ensure creating valid object
    artist = input("Enter the artist:")
    songs = input("Enter the songs:")
    streams = input("Enter the streams:")
    
    # Initialize User object before date
    album = Album(name=name, 
                  artist=artist,
                  songs=songs,
                  streams=streams
                )
    
    # create user.dob, fail with today as dob
    release = input("Enter the release date 'YYYY-MM-DD'")
    try:
        album.releaase = datetime.strptime(release, '%Y-%m-%d').date()
    except ValueError:
        album.release = datetime.today()
        print(f"Invalid date {release} require YYYY-mm-dd, date defaulted to {album.release}")
           
    # write object to database
    with app.app_context():
        try:
            object = album.update()
            print("Updated\n", object.read())
        except:  # error raised if object not created
            print("Unknown error name {name}")
        
update()

Found
 {'id': 1, 'name': 'ye', 'artist': 'Kanye West', 'songs': '7', 'streams': '7', 'release': '01-01-2018', 'age': 5}


TypeError: __init__() got an unexpected keyword argument 'id'

In [ ]:
# delete function
# Find the album by name
def delete():
    name = input("(delete)which album to delete: ")
    album = find_by_name(name)

    if album is None:
        print(name + " not found")
        return

    with app.app_context():
        try:
            album.delete()
            print(name + " has been deleted")
        except:
            print("could not delete " + name)

delete()


Afterlyfe has been deleted


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.